In [1]:
import numpy as np
import os
import pandas as pd
import pickle
import re

- If Neiss_data.pckl exists read it into the dataframe.  Otherwise, read all the neiss1999 ... neiss2018 files into a dataframe and save that dataframe as neiss_data.pckl

In [2]:
neiss_pathname = os.getcwd() + '/data/NEISS'

def readNeissPcklFile(pckl_fname):
    if os.path.exists(pckl_fname):
        print("Reading {}  ... ".format(pckl_fname), end="")
        neiss_df = pickle.load( open( pckl_fname, "rb" ) )
        print("done!")
        return neiss_df
    return None

def readNeissOriginalData(neiss_pathname, expression):
    neiss_df = pd.DataFrame()
    for subdir, dirs, files in os.walk(neiss_pathname):
        files.sort()
        filtered_fnames = filter(lambda fname: re.search(expression, fname), files)

        for index, file in enumerate(filtered_fnames):
            fname = '{}/{}'.format(subdir, file)
            print("Reading {}  ... ".format(fname))

            neiss_df1 = pd.read_excel(fname)
            neiss_df = pd.concat([neiss_df, neiss_df1])
    return neiss_df1

def saveNeissOriginalDataToPcklFile(df, pckl_fname):
    with open(pckl_fname, 'wb') as f:
        pickle.dump(df, f)
        print("Saved", pckl_fname)
        
pckl_fname_neiss = neiss_pathname + '/neiss_data.pckl'
neiss_df = readNeissPcklFile(pckl_fname_neiss)
if neiss_df.empty:
    neiss_df = readNeissOriginalData(neiss_pathname, '^neiss\d{4}.xlsx$')
    saveNeissOriginalDataToPcklFile(neiss_df, pckl_fname_neiss)
neiss_df

# 

Reading /Users/mcorbett/Boston University/MET CS677 - Data Science with Python/Project/data/NEISS/neiss_data.pckl  ... done!


,CPSC_Case_Number,Treatment_Date,Age,Sex,Race,Other_Race,Body_Part,Diagnosis,Other_Diagnosis,Disposition,Location,Fire_Involvement,Product_1,Product_2,Narrative_1,Narrative_2,Stratum,PSU,Weight
0,180106119,2018-01-01,57,2,2,NaN,79,53,NaN,1,1,0,4074,611,57 YO F CAREGIVER WAS BATHING PT WHEN PT SLID ...,NaN,S,71,70.9703
1,180106122,2018-01-01,70,1,1,NaN,76,59,NaN,1,0,0,4057,0,70 YO M VOMITTING AND PASSED OUT HITTING HEAD ...,NaN,S,71,70.9703
2,180106156,2018-01-01,75,1,1,NaN,75,59,NaN,1,1,0,1807,0,75YOM-SCALP LAC-FELL HITTING HEAD ON CONCRETE ...,NaN,S,11,70.9703
3,180106157,2018-01-01,34,1,1,NaN,92,59,NaN,1,1,0,474,0,34YOM-FINGER LAC-CUT ON A BROKEN COFFEE MUG-@ ...,NaN,S,11,70.9703
4,180106178,2018-01-01,33,1,3,HISPANIC,92,59,NaN,1,1,0,464,0,33YOM WAS CUTTING LIMES WITH A KNIFE AND CUT L...,NaN,S,46,70.9703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361662,190404184,2018-12-23,65,2,0,NaN,92,59,NaN,1,1,0,464,0,65YOF CUT FINGER WITH KITCHEN KNIFE. DX: FINGE...,NaN,V,21,17.5136
361663,190404185,2018-12-23,9,1,0,NaN,80,57,NaN,4,1,0,4074,0,9YOM RUNNING WITH BEAN BAG CHAIR AND FELL ONTO...,NaN,V,21,17.5136
361664,190404186,2018-12-23,35,1,0,NaN,92,59,NaN,1,1,0,464,0,35YOM CUT FINGER WITH KITCHEN KNIFE. DX: FINGE...,NaN,V,21,17.5136
361665,190404188,2018-12-23,43,1,0,NaN,83,64,NaN,1,1,0,679,0,43YOM ROLLED FOOT WHEN GETTING UP FROM COUCH. ...,NaN,V,21,17.5136


In [4]:
def generateCodeIdTranslator(df, categories):
    indices = pd.MultiIndex.from_product(
        categories, names=['Category', 'Function'])   # Uses the dot-product to create indices

    dfCodeTranslator = pd.DataFrame(index = indices, columns = [])
    for index, category in enumerate(categories[0]):
        unique_codes = np.sort(df[category].unique())

        # Generate the id to code dictionary and insert it into a temporary dataframe
        idToCode = {index : value for index, value in enumerate(unique_codes)}
        dfIdToCode = pd.DataFrame(index=[[category], ['idToCode']], columns=['translator'])
        dfIdToCode.loc[(category, 'idToCode'), 'translator'] = [idToCode] # for some reason the dict has to be in a list

        # Generate the code to id dictionary and insert it into a temporary dataframe
        codeToId = {v: k for k, v in idToCode.items()}
        dfCodeToId = pd.DataFrame(index=[[category], ['codeToId']], columns=['translator'])
        dfCodeToId.loc[(category, 'codeToId'), 'translator'] = [codeToId] # for some reason the dict has to be in a list

        # Add the two temporary dataframes to the dfCodeTranslator dataframe
        dfCodeTranslator = pd.concat([dfCodeTranslator, dfIdToCode, dfCodeToId], sort=False)
    dfCodeTranslator.dropna(inplace=True)
    dfCodeTranslator = dfCodeTranslator.unstack(level=0)
    dfCodeTranslator = dfCodeTranslator.stack()
    return dfCodeTranslator


def saveCodeTranslatorToPckl(df, pckl_fname):
    '''
    Save the code/id translator dataframe to a pckl file.
    
    Args:
      df (pd.Dataframe):     The dataframe to be written to the pckl file
      pckl_fname (str):      The fully qualified pathname to the pckl file
    '''
    with open(pckl_fname, 'wb') as f:
        pickle.dump(df, f)
        print("Saved", pckl_fname)
        

categories = [
        ['Sex', 'Race', 'Body_Part', 'Diagnosis', 'Disposition', 'Location',
         'Fire_Involvement', 'Product_1', 'Product_2', 'PSU', 'Stratum'],

        ['codeToId', 'idToCode']
    ]
dfCodeTranslator = generateCodeIdTranslator(neiss_df, categories)
print(dfCodeTranslator)


pckl_fname_code_translator = neiss_pathname + '/code_translator_df.pckl'
saveCodeTranslatorToPckl(dfCodeTranslator, pckl_fname_code_translator)





TypeError: generateCodeIdTranslator() missing 1 required positional argument: 'categories'